In [1]:
import pandas as pd

## Test Cleanup

In [6]:
tests_url = 'https://opendata.arcgis.com/datasets/c3c0dc1891454ad5a49b7d9a3438471a_0.csv'       

In [10]:
df_tests = pd.read_csv(tests_url, parse_dates=['Date_Completed'])

In [11]:
df_tests.tail()

,Date_Completed,ObjectId,Commercial_Tests,Commercial_Tests__Cumulative_,ASPHL_Tests,ASPHL_Tests__Cumulative_,Hosp_Fac_Tests,Hosp__Fac__Tests__Cumulative_,All_Tests,All_Tests__Cumulative_
153,2020-08-02 12:00:00+00:00,154,769,58993,1015,92296,933,94518,2717,245807
154,2020-08-03 12:00:00+00:00,155,1122,60115,1516,93812,4204,98722,6842,252649
155,2020-08-04 12:00:00+00:00,156,855,60970,1412,95224,2363,101085,4630,257279
156,2020-08-05 12:00:00+00:00,157,1167,62137,1468,96692,1658,102743,4293,261572
157,2020-08-06 12:00:00+00:00,158,1732,63869,1711,98403,3836,106579,7279,268851


In [12]:
tests_map = {
    'Date_Completed' : 'date',
    'ASPHL_Tests' : 'tests_ASPHL',
    'Commercial_Tests' : 'tests_commercial',
    'Hosp_Fac_Tests' : 'tests_POC', 
    'All_Tests' : 'tests_total',
}

df_tests = df_tests[list(tests_map.keys())].copy()
df_tests = df_tests.rename(columns=tests_map)
df_tests['date'] = pd.to_datetime(df_tests['date']).dt.date
# df_tests.to_csv('../data/parsed/tests.csv', index=False)

In [13]:
df_tests.tail()

,date,tests_ASPHL,tests_commercial,tests_POC,tests_total
153,2020-08-02,1015,769,933,2717
154,2020-08-03,1516,1122,4204,6842
155,2020-08-04,1412,855,2363,4630
156,2020-08-05,1468,1167,1658,4293
157,2020-08-06,1711,1732,3836,7279


## Case Data

In [138]:
case_url = 'https://opendata.arcgis.com/datasets/c1b6c31d09b44c33962570950456feea_0.csv'

In [139]:
df_cases = pd.read_csv(case_url, parse_dates=['OnsetDate'])

In [141]:
df_cases.head(10)

,County_Name,County_Code,Econ_Name,Econ_Code,BHR_Name,BHR_Code,OnsetDate,SpecificAcquisition,OnsetDateReplaced,Resident,Occurrence,FID
0,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-12 12:00:00+00:00,Travel,0,Y,Y,1
1,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-14 12:00:00+00:00,Travel,0,Y,Y,2
2,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-07 12:00:00+00:00,Travel,0,Y,Y,3
3,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-15 12:00:00+00:00,Travel,0,Y,Y,4
4,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-14 12:00:00+00:00,Travel,0,Y,Y,5
5,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-17 12:00:00+00:00,Travel,0,Y,Y,6
6,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-15 12:00:00+00:00,Travel,0,Y,Y,7
7,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-17 12:00:00+00:00,Community,0,Y,Y,8
8,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-19 12:00:00+00:00,Community,0,Y,Y,9
9,Anchorage Municipality,20,Anchorage,7,Anchorage Municipality,1,2020-03-11 12:00:00+00:00,Travel,0,Y,Y,10


In [142]:
cases_map ={
        'OnsetDate' : 'date',
        'Econ_Name' : 'region',
        'Resident' : 'resident',
    }

df_cases = df_cases[list(cases_map.keys())].copy()
df_cases = df_cases.rename(columns=cases_map)
df_cases['date'] = pd.to_datetime(df_cases['date']).dt.date

In [143]:
df_res_cases = df_cases[df_cases['resident']=='Y'].copy()
df_res_cases = df_res_cases.groupby(['date', 'region']).count().copy().reset_index()

In [144]:
df_res_cases = pd.pivot_table(df_res_cases, columns=['region'], values='resident', index='date').fillna(0)

In [145]:
regions = df_res_cases.columns.tolist()

In [146]:
df_res_cases['Statewide'] = df_res_cases[regions].sum(axis=1)

In [147]:
df_res_cases.tail()

region,Anchorage,Gulf Coast,Interior,Matanuska-Susitna,Northern,Southeast,Southwest,Statewide
date,,,,,,,,
2020-08-02,40.0,0.0,4.0,4.0,4.0,5.0,2.0,59.0
2020-08-03,42.0,0.0,4.0,6.0,3.0,2.0,2.0,59.0
2020-08-04,38.0,2.0,4.0,3.0,4.0,4.0,1.0,56.0
2020-08-05,25.0,3.0,7.0,1.0,2.0,2.0,0.0,40.0
2020-08-06,29.0,5.0,9.0,3.0,1.0,3.0,2.0,52.0


In [148]:
df_res_cases = pd.melt(df_res_cases.reset_index(), id_vars='date')
df_res_cases.columns = ['date', 'region', 'resident']

In [149]:
df_nonres_cases = df_cases[df_cases['resident']=='N'].copy()
df_nonres_cases = df_nonres_cases.groupby(['date', 'region']).count().copy().reset_index()

In [150]:
df_nonres_cases = pd.pivot_table(df_nonres_cases, columns=['region'], values='resident', index='date').fillna(0)

In [151]:
regions = df_nonres_cases.columns.tolist()

In [152]:
df_nonres_cases['Statewide'] = df_nonres_cases[regions].sum(axis=1)

In [153]:
df_nonres_cases = pd.melt(df_nonres_cases.reset_index(), id_vars='date')

In [154]:
df_nonres_cases.columns = ['date', 'region', 'nonresident']

In [157]:
df_cases = df_res_cases.merge(df_nonres_cases, left_on=['date','region'], right_on=['date', 'region'], how='outer').fillna(0)

In [158]:
df_cases['total'] = df_cases[['resident', 'nonresident']].sum(axis=1)
# df_cases['date'] = pd.to_datetime(df_cases['date'])

In [159]:
df_cases.tail()

,date,region,resident,nonresident,total
1312,2020-08-03,Unknown,0.0,2.0,2.0
1313,2020-08-04,Unknown,0.0,0.0,0.0
1314,2020-08-05,Unknown,0.0,0.0,0.0
1315,2020-08-06,Unknown,0.0,0.0,0.0
1316,2020-05-08,Statewide,0.0,1.0,1.0


In [162]:
df_cases[df_cases['region']=='Statewide']['resident'].sum()+78

3614.0